In [1]:
import numpy as np
np.random.seed(7)

class SimpleQueue:
    """Define a class called SimpleQueue"""

    def __init__(self, num_delays_required):
        """Initialization function"""

        # Specify input parameters.
        self.mean_interarrival = 2.000
        self.mean_service = 0.500
        self.num_delays_required = num_delays_required

        # Initialize state variables.
        self.server_status = 0  # 0 is idle and 1 is busy.
        self.num_in_q = 0
        self.time_arrival = []  # List for times of arrival of customers.
        self.time_last_event = 0.0

        # Initialize statistical counters.
        self.num_custs_delayed = 0
        self.total_of_delays = 0.0
        self.area_num_in_q = 0.0
        self.area_server_status = 0.0

        # Initialize simulation clock.
        self.sim_time = 0.0

        # Initialize event list.
        self.time_next_event = [0, 0, 0]   # The list self.time_next_event has 3 entries, with index 0 not used.
        self.time_next_event[1] = self.sim_time + self.expon(self.mean_interarrival)
        self.time_next_event[2] = float('inf')

        # Initialize other variables.
        self.num_events = 2

    def main(self):
        """Main function."""


        # Run the simulation while more delays are needed.
        while self.num_custs_delayed < self.num_delays_required:

            # Determine the next event type.
            self.timing()

            # Update the time-average statistical counters.
            self.update_time_avg_stats()

            # Invoke the appropriate event function.
            if self.next_event_type == 1:
                self.arrive()
            elif self.next_event_type == 2:
                self.depart()
            else:
                break

        # Invoke the report generator.
        self.report()

    def timing(self):
        """Timing function."""

        # Initialize variables needed to search event list for minimum time.
        self.min_time_next_event = float('inf')
        self.next_event_type = 0

        # Determine the event type of the next event to occur.
        for i in range(1,self.num_events+1):   # Range considers i = 1, 2,..., num.events.
            if self.time_next_event[i] < self.min_time_next_event:
                self.min_time_next_event = self.time_next_event[i]
                self.next_event_type = i

        # Check to see whether all entries in the event list have 'infinite' values.
        if self.next_event_type == 0:
            # Stop the simulation.
            raise Exception("All entries in the event list have infinite values, so stop the simulation.")

        # The event list has an entry with a finite value, so advance the simulation clock.
        self.sim_time = self.min_time_next_event

    def arrive(self):
        """Arrive event function."""

        # Schedule next arrival.
        self.time_next_event[1] = self.sim_time + self.expon(self.mean_interarrival)

        # Check to see whether the server is busy.
        if self.server_status == 1:
            # Server is busy, so increment the number of customers in queue and store the
            # time of arrival of the arriving customer at the end of the list self.time_arrival.
            self.num_in_q += 1
            self.time_arrival.append(self.sim_time)
        else:
            # Server is idle, so arriving customer has a delay of zero.  (The following two statements
            # are for program clarity and do not affect the simulation results.)
            self.delay = 0.0
            self.total_of_delays += self.delay

            # Increment the number of customers delayed and make server busy.
            self.num_custs_delayed += 1
            self.server_status = 1

            # Schedule a departure.
            self.time_next_event[2] = self.sim_time + self.expon(self.mean_service)

    def depart(self):
        """Depart event function."""

        # Check to see whether the queue is empty.
        if self.num_in_q == 0:
            # The queue is empty, so make the server idle and
            # eliminate the departure event from consideration.
            self.server_status = 0
            self.time_next_event[2] = float('inf')
        else:
            # The queue is nonempty, so decrement the number in queue.
            self.num_in_q -= 1

            # Compute the delay of the customer who is beginning
            # service and update the total delay accumulator.
            self.delay = self.sim_time - self.time_arrival[0]   # The index of the first location in a list is 0.
            self.total_of_delays += self.delay

            # Increment the number of customers delayed and schedule a departure.
            self.num_custs_delayed += 1
            self.time_next_event[2] = self.sim_time + self.expon(self.mean_service)

            # Delete the customer who was first in queue.
            del self.time_arrival[0]

    def report(self):
        """Report generator function.
        ""
        print 
        print("Average delay in queue: {:.3f} minutes".format(self.total_of_delays / self.num_custs_delayed))
        print("Average number in queue: {:.3f}".format(self.area_num_in_q / self.sim_time))
        print("Server utilization: {:.3f}".format(self.area_server_status / self.sim_time))
        print("Time simulation ended: {:.3f} minutes".format(self.sim_time))
        print("-" * 40)
        """ 
    def update_time_avg_stats(self):
        """Function to compute time-average statistics."""

        # Compute time since last event and update time-of-last-event marker.
        self.time_since_last_event = self.sim_time - self.time_last_event
        self.time_last_event = self.sim_time

        # Update area under number-in-queue variable.
        self.area_num_in_q += self.num_in_q * self.time_since_last_event

        # Update area under the server-busy indicator variable.
        self.area_server_status += self.server_status * self.time_since_last_event

    def expon(self, mean):
        """Function to generate exponential random variates."""

        return -mean * np.log(np.random.uniform(0, 1))

# Create an instance of class MM1_Queue.
m = SimpleQueue(100000)

# Call main function to start the simulation.
m.main()